In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q langchain langchain-groq chromadb sentence-transformers datasets spacy python-dotenv accelerate peft bitsandbytes
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.6 MB/s e

In [3]:
!pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [4]:
# %% [Import Libraries]
import os
import spacy
import chromadb
import pandas as pd
from datasets import load_dataset
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from peft import LoraConfig, get_peft_model
from google.colab import userdata  # For secrets management


In [5]:
from google.colab import userdata
userdata.get('GROQ_API_KEY')

'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'

In [6]:
# %% [Install Required Libraries]
!pip install -q datasets  # Install Hugging Face datasets library

# %% [Load Dataset from Hugging Face]
from datasets import load_dataset

def load_empathic_data():
    """Load and split the Empathetic Dialogues dataset directly from Hugging Face"""
    dataset = load_dataset("empathetic_dialogues")
    return dataset["train"], dataset["validation"], dataset["test"]

# Load all splits
train_data, valid_data, test_data = load_empathic_data()

# %% [Verify Data Loading]
print("Dataset structure:")
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(valid_data)}")
print(f"Test samples: {len(test_data)}")

print("\nSample training example:")
print(f"Context: {train_data[0]['context']}")
print(f"Utterance: {train_data[0]['utterance']}")
print(f"Speaker: {train_data[0]['speaker_idx']}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.15k [00:00<?, ?B/s]

empathetic_dialogues.py:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

The repository for empathetic_dialogues contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/empathetic_dialogues.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/76673 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10943 [00:00<?, ? examples/s]

Dataset structure:
Training samples: 76673
Validation samples: 12030
Test samples: 10943

Sample training example:
Context: sentimental
Utterance: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people_comma_ we felt like the only people in the world.
Speaker: 1


In [7]:
dataset = load_dataset("empathetic_dialogues")  # Direct download from HF hub


In [8]:
print(f"Training samples: {len(train_data)}")  # Shows 76,888 examples
print(f"Validation samples: {len(valid_data)}")  # Shows 6,874 examples
print(f"Test samples: {len(test_data)}")  # Shows 7,097 examples

Training samples: 76673
Validation samples: 12030
Test samples: 10943


In [9]:
# %% [Text Preprocessing - Optimized for Colab]
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def preprocess_text(text):
    doc = nlp(text)
    return " ".join([
        token.lemma_.lower() for token in doc
        if not token.is_stop and token.is_alpha and len(token) > 2
    ])

In [10]:
!pip install -U langchain-community  # Correct pip install command
from langchain_community.vectorstores import Chroma  # Correct import statement

In [11]:
# %% [Sentiment Analysis Model]
class SentimentAnalyzer:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.model = AutoModelForSequenceClassification.from_pretrained(
            "bert-base-uncased",
            num_labels=5,  # Assuming 5 sentiment labels: anger, sadness, anxiety, joy, neutral
        )

        # Move model to GPU if available
        if torch.cuda.is_available():
            self.model.to("cuda")

        self.labels = ["anger", "sadness", "anxiety", "joy", "neutral"]

    def predict_mood(self, text):
        # Preprocess text and tokenize
        inputs = self.tokenizer(preprocess_text(text), return_tensors="pt", truncation=True)

        # Move inputs to GPU if available
        if torch.cuda.is_available():
            inputs = inputs.to("cuda")

        # Perform inference
        with torch.no_grad():  # Disable gradient calculation during inference
            try:
                outputs = self.model(**inputs)
            except RuntimeError as e:
                if "out of memory" in str(e):
                    print("CUDA out of memory. Trying to clear cache...")
                    torch.cuda.empty_cache()
                    outputs = self.model(**inputs)  # Retry inference
                else:
                    raise e  # Re-raise other exceptions

        # Get predicted label
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_label_index = torch.argmax(probs).item()  # Get index of predicted label
        return self.labels[predicted_label_index]

In [12]:
# %% [Knowledge Base Setup]
def create_knowledge_base():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    chroma_client = chromadb.PersistentClient(path="/kaggle/working/mindmate_db")

    return Chroma(
        collection_name="mental_health_knowledge",
        embedding_function=embeddings,
        client=chroma_client
    )

vector_store = create_knowledge_base()

# Add processed data to vector store
#processed_data = [preprocess_text(f"{ex['context']} {ex['utterance']}") for ex in train_data] # No longer needed here since we're processing in the next cell
#vector_store.add_texts(processed_data[:5000])  # Adjust based on memory # No longer needed here

# %%
from tqdm import tqdm
import pandas as pd # Import pandas

print("Processing data...")
# Convert the Hugging Face Dataset to a Pandas DataFrame first
df = pd.DataFrame(train_data[:5000])
processed_data = [preprocess_text(f"{ex['context']} {ex['utterance']}") for ex in tqdm(df.to_dict(orient='records'))]  # Convert DataFrame to list of dictionaries
print("Storing embeddings...")
vector_store.add_texts(processed_data) # Now vector_store is defined
vector_store.persist # Persist the data

<ipython-input-12-3ef0e5e072a2>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

<ipython-input-12-3ef0e5e072a2>:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(


Processing data...


100%|██████████| 5000/5000 [00:41<00:00, 121.88it/s]


Storing embeddings...


<bound method Chroma.persist of <langchain_community.vectorstores.chroma.Chroma object at 0x7dd1541ec350>>

In [13]:
# %% [Main Pipeline - Colab Optimized]
class MindMate:
    def __init__(self):
        self.sentiment_analyzer = SentimentAnalyzer()
        self.llm = ChatGroq(
            temperature=0.7,
            model_name="mixtral-8x7b-32768",
            groq_api_key=userdata.get('GROQ_API_KEY')
        )
        self.mood_history = []
        self.crisis_keywords = {"suicide", "kill myself", "self harm", "abuse"}

        self.prompt_template = PromptTemplate(
            input_variables=["context", "message", "sentiment"],
            template="""[SYSTEM] You are a mental health assistant. Current emotion: {sentiment}
            Context: {context}
            [USER] {message}
            [ASSISTANT]"""
        )

    def generate_response(self, user_input):
        if any(kw in user_input.lower() for kw in self.crisis_keywords):
            return self.handle_crisis(), "crisis"

        sentiment = self.sentiment_analyzer.predict_mood(user_input)
        self.mood_history.append(sentiment)

        docs = vector_store.similarity_search(user_input, k=2)
        context = "\n".join([doc.page_content for doc in docs])

        chain = LLMChain(llm=self.llm, prompt=self.prompt_template)
        response = chain.run(context=context, message=user_input, sentiment=sentiment)

        return response, sentiment

    def handle_crisis(self):
        return """🚨 Immediate Help Resources:
1. National Suicide Prevention Lifeline (US): 1-800-273-8255
2. Crisis Text Line: Text HOME to 741741
3. International Emergency: 112 or your local emergency number"""

# %% [Colab Chat Interface]
def colab_chat():
    print("MindMate: Hello! I'm here to listen. Type 'exit' to end the chat.\n")
    mindmate = MindMate()

    while True:
        try:
            user_input = input("You: ")
            if user_input.lower() in ["exit", "quit"]:
                break

            response, sentiment = mindmate.generate_response(user_input)
            print(f"\nMindMate: {response}")
            print(f"Detected Emotion: {sentiment.capitalize()}\n")

        except KeyboardInterrupt:
            print("\nChat session ended.")
            break

# Run in Colab
if __name__ == "__main__":
    colab_chat()

MindMate: Hello! I'm here to listen. Type 'exit' to end the chat.



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


You: I am sad


<ipython-input-13-de9ff788694a>:31: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=self.llm, prompt=self.prompt_template)
<ipython-input-13-de9ff788694a>:32: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(context=context, message=user_input, sentiment=sentiment)



MindMate: I'm really sorry that you're feeling this way, but I'm unable to provide the help that you need. It's really important to talk things over with someone who can, though, such as a mental health professional or a trusted person in your life.
Detected Emotion: Neutral


Chat session ended.


In [14]:
!pip install -q fastapi nest-asyncio pyngrok uvicorn gradio python-multipart


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 103.3 MB/s eta 0:00:00


In [15]:
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get('NGROK_TOKEN')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [16]:
# %% [FastAPI Backend]
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import torch
import uvicorn
from typing import Dict

app = FastAPI(title="MindMate API", version="1.0")

In [17]:
# CORS Configuration
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Request Models
class AnalysisRequest(BaseModel):
    text: str

class ChatRequest(BaseModel):
    message: str
    session_id: str = "default"


In [18]:
!pip install -q langchain langchain-experimental chromadb sentence-transformers datasets spacy python-dotenv accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 12.7 MB/s eta 0:00:00


In [19]:
 !apt-get install

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.


In [20]:
!pip install -q langchain-experimental
# Import the correct module or function from the correct path
from langchain_groq import ChatGroq# Import from langchain.utilities instead of langchain_experimental.groq

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [22]:
# %% [FastAPI Backend]
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import torch
import uvicorn
from typing import Dict

app = FastAPI(title="MindMate API", version="1.0")
# %%
# CORS Configuration
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Request Models
class AnalysisRequest(BaseModel):
    text: str

class ChatRequest(BaseModel):
    message: str
    session_id: str = "default"

# %%

class ChatRequest(BaseModel):
    message: str
    session_id: str = "default"
# Define the MindMate class (Assuming this is the definition from a previous cell in your notebook)
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from google.colab import userdata
# ... other imports
groq_api_key = userdata.get('GROQ_API_KEY')

class MindMate:
    def __init__(self):
        self.sentiment_analyzer = SentimentAnalyzer()
        self.llm = ChatGroq(
            temperature=0.7,
            model_name="mixtral-8x7b-32768",
            groq_api_key=groq_api_key
        )
        self.mood_history = []
        self.crisis_keywords = {"suicide", "kill myself", "self harm", "abuse"}

        self.prompt_template = PromptTemplate(
            input_variables=["context", "message", "sentiment"],
            template="""[SYSTEM] You are a mental health assistant. Current emotion: {sentiment}
            Context: {context}
            [USER] {message}
            [ASSISTANT]"""
        )

    def generate_response(self, user_input):
        if any(kw in user_input.lower() for kw in self.crisis_keywords):
            return self.handle_crisis(), "crisis"

        sentiment = self.sentiment_analyzer.predict_mood(user_input)
        self.mood_history.append(sentiment)

        docs = vector_store.similarity_search(user_input, k=2)
        context = "\n".join([doc.page_content for doc in docs])

        chain = LLMChain(llm=self.llm, prompt=self.prompt_template)
        response = chain.run(context=context, message=user_input, sentiment=sentiment)

        return response, sentiment

    def handle_crisis(self):
        return """🚨 Immediate Help Resources:
1. National Suicide Prevention Lifeline (US): 1-800-273-8255
2. Crisis Text Line: Text HOME to 741741
3. International Emergency: 112 or your local emergency number"""

mindmate = MindMate()  # Your existing MindMate class

@app.post("/analyze")
async def analyze_sentiment(request: AnalysisRequest):
    try:
        sentiment = mindmate.sentiment_analyzer.predict_mood(request.text)
        return {"sentiment": sentiment, "text": request.text}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/chat")
async def chat_endpoint(request: ChatRequest):
    try:
        response, sentiment = mindmate.generate_response(request.message)
        return {
            "response": response,
            "sentiment": sentiment,
            "session_id": request.session_id
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# ... the rest of the code


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
from fastapi import FastAPI, HTTPException, Depends # Import Depends
from fastapi.middleware.cors import CORSMiddleware
from fastapi.security import APIKeyHeader
from pydantic import BaseModel
import torch
import uvicorn
from typing import Dict

# ... (rest of your code) ...

security = APIKeyHeader(name="X-API-Key")

@app.post("/chat")
async def secure_chat(request: ChatRequest, api_key: str = Depends(security)):
    if api_key != os.getenv("API_SECRET"):
        raise HTTPException(status_code=403)
    # ... existing code ...

In [24]:
#!python -m uvicorn main:app --reload --port 8000

In [25]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# Initialize your existing MindMate class
mindmate = MindMate()  # Your previously defined class

app = FastAPI()

class ChatRequest(BaseModel):
    message: str
    session_id: str = "default"

@app.post("/chat")
async def chat_endpoint(request: ChatRequest):
    try:
        response, sentiment = mindmate.generate_response(request.message)
        return {
            "response": response,
            "sentiment": sentiment,
            "session_id": request.session_id
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Remove the if __name__ == "__main__": block and the call to uvicorn.run
# Instead, run the following command in a separate terminal to start Uvicorn:
# uvicorn main:app --reload --host 0.0.0.0 --port 8000

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
!pip install prometheus-fastapi-instrumentator

In [27]:
from prometheus_fastapi_instrumentator import Instrumentator
Instrumentator().instrument(app).expose(app)

In [28]:
!pip install gradio

In [31]:
!pip install groq

In [34]:
!pip install -q gradio>=4.22.0

In [37]:
import gradio as gr

In [38]:
gr.Chatbot(value=[{"role": "system", "content": "I am MindMate Pro, your mental health assistant."}], type="messages")

In [4]:
# %% [Install Dependencies]
!pip install -q gradio>=4.22.0 pandas

# %% [Gradio Interface]
import gradio as gr
import pandas as pd
import logging

def create_interface():
    with gr.Blocks(theme=gr.themes.Soft(), title="MindMate Pro") as demo:
        # Header Section
        gr.Markdown("""
        # 🧠 MindMate Pro - Advanced Mental Health Assistant
        *Your 24/7 AI-powered mental health companion*
        """)

        # Main Chat Interface
        with gr.Row():
            # Left Panel - Chat Interface
            with gr.Column(scale=3):
                chatbot = gr.Chatbot(
                    label="Conversation History",
                    avatar_images=(None, None),
                    height=500,
                    type="messages"
                )
                input_box = gr.Textbox(
                    placeholder="How are you feeling today?",
                    label="Your Message",
                    lines=2
                )
                with gr.Row():
                    submit_btn = gr.Button("Send", variant="primary")
                    clear_btn = gr.Button("Clear History")

                gr.Examples(
                    examples=[
                        ["I've been feeling anxious"],
                        ["I can't sleep at night"],
                        ["I'm struggling with relationships"]
                    ],
                    inputs=input_box,
                    label="Common Concerns"
                )

            # Right Panel - Analytics
            with gr.Column(scale=2):
                with gr.Tab("Emotion Analysis"):
                    emotion_graph = gr.LinePlot(
                        label="Mood History",
                        show_label=True,
                        interactive=False,
                        x="Time",
                        y="Mood"
                    )
                    emotion_summary = gr.Dataframe(
                        headers=["Emotion", "Count"],
                        label="Emotion Distribution"
                    )

                with gr.Tab("Safety Tools"):
                    gr.Markdown("""
                    **Immediate Help Resources:**
                    - National Suicide Prevention Lifeline: 1-800-273-8255
                    - Crisis Text Line: Text HOME to 741741
                    - International Emergency: 112
                    """)

        # Hidden Components
        session_state = gr.State(value={"mood_history": []})
        error_html = gr.HTML()

        # Event Handlers
        def process_message(message, history, session):
            try:
                # Replace with actual model call
                response = "This is a sample response"
                sentiment = "neutral"

                if "sad" in message.lower():
                    sentiment = "sadness"
                elif "angry" in message.lower():
                    sentiment = "anger"

                session["mood_history"].append(sentiment)

                # Create plot data
                plot_data = pd.DataFrame({
                    "Time": range(len(session["mood_history"])),
                    "Mood": session["mood_history"]
                })

                # Create emotion summary
                emotion_df = pd.DataFrame({
                    "Emotion": session["mood_history"]
                }).value_counts().reset_index()
                emotion_df.columns = ["Emotion", "Count"]

                # Format messages correctly
                new_messages = history + [
                    {"role": "user", "content": message},
                    {"role": "assistant", "content": response}
                ]

                return {
                    chatbot: new_messages,
                    emotion_graph: plot_data,
                    emotion_summary: emotion_df,
                    error_html: ""
                }
            except Exception as e:
                logging.error(f"Error: {str(e)}")
                return {
                    error_html: f"<div style='color: red; padding: 10px;'>Error: {str(e)}</div>",
                    chatbot: history,
                    emotion_graph: None,
                    emotion_summary: None
                }

        def clear_history():
            empty_plot = pd.DataFrame(columns=["Time", "Mood"])
            empty_summary = pd.DataFrame(columns=["Emotion", "Count"])
            return (
                {"session_state": {"mood_history": []}},
                [],
                empty_plot,
                empty_summary
            )

        # Component Links
        input_box.submit(
            process_message,
            [input_box, chatbot, session_state],
            [chatbot, emotion_graph, emotion_summary, error_html]
        )
        submit_btn.click(
            process_message,
            [input_box, chatbot, session_state],
            [chatbot, emotion_graph, emotion_summary, error_html]
        )
        clear_btn.click(
            clear_history,
            outputs=[session_state, chatbot, emotion_graph, emotion_summary]
        )

    return demo

# %% [Launch Interface]
if __name__ == "__main__":
    interface = create_interface()
    interface.launch(server_port=None, server_name="0.0.0.0")  # Let Gradio find an available port

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://227adfa774bf69a593.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
